# MIRI Low-Resolution Spectroscopy Mode Examples

Author: Jonathan Aguilar (jaguilar@stsci.edu)

This notebook demonstrates the classes for visualizing the three MIRI LRS Spectroscopy modes - Slit, Slitless, and Wide-Field Slitless. We will see how to adjust the observing parameters to avoid overlapping traces while keeping sources on the detector. For a more general introduction to WHIPPOT, see [WHIPPOT_tutorial](WHIPPOT_tutorial).

The user provides the RA and Dec coordinates for the target acquisition (ACQ) star and science (SCI) star, and a telescope roll angle. These tools will then compute a) the "Special Requirement Offset" X and Y values needed to place the SCI star at the reference position, and b) the final coordinates of the SCI and ACQ stars. Coordinates are given in IDL (https://jwst-docs.stsci.edu/jwst-observatory-hardware/jwst-target-observability-and-observatory-coordinate-system/jwst-instrument-ideal-coordinate-systems#gsc.tab=0).

The user can also provide a list of extra targets in the field in the format specified below, and their IDL coordinates will also be computed. As such, this notebook is useful not just for planning target acquisition, but also for predicting to high precision the final positions of astronomical bodies in the field of view.

## Setup ##

  1. Navigate in a terminal to the directory where you have downloaded the module. This should contain the `setup.py` file.
  2. Activate your desired python environment.
  3. Run `pip install .` (or `pip install -e.` for developing)
  4. Copy this notebook to your working directory, and run your jupyter kernel in the appropriate environment.

In [ ]:
# for development purposes
%load_ext autoreload
%autoreload 2

In [ ]:
# for your system, choose an appropriate plotting backend to make pop-up window plots
# (currently, inline plots show up in the log but not directly in the notebook).
# %matplotlib throws an error if you run it first, so do inline first.
%matplotlib inline
%matplotlib osx

In [ ]:
from whippot import whippot_tools
# if the library is not installed replace the line above with the line below 
# when `whippot_tools.py` is in the same folder as the notebook
# import whippot_tools

## Which LRS observing mode is right for our field?

The source coordinates below are taken from a real WFSS commissioning activity, where we needed to check that the traces wouldn't overlap. In this notebook, we're going to imagine that we want to get simultaneous spectra of all these sources, but we aren't sure which mode is appropriate (just go with me here). WHIPPOT can help us investigate.

In [ ]:
# Here are the 5 sources we are interested in
sources = {
    'SCI': whippot_tools.SkyCoord("05 24 20.7552 -70 05 1.60", frame='icrs', unit=("hourangle","degree")),
    'a': whippot_tools.SkyCoord("05 24 26.33969 -70 05 22.3545", frame='icrs', unit=("hourangle","degree")),
    'b': whippot_tools.SkyCoord("05 24 28.67861 -70 05 24.4484", frame='icrs', unit=("hourangle","degree")),
    'c': whippot_tools.SkyCoord("05 24 36.22460 -70 05 28.1876", frame='icrs', unit=("hourangle","degree")),
    'd': whippot_tools.SkyCoord("05 24 25.608 -70 05 01.66", frame='icrs', unit=("hourangle","degree")),
}

## MIRI LRS Modes

The `miri_lrs_tools` class contains four class definitions - one base class, and then one class that is specific to each mode. The base class `MiriLRS` is a subclass of `ComputePositions`, that contains methods common to all LRS modes. The three mode-specific classes are in turn subclasses of `MiriLRS`:
- `MiriLRSSlit`
- `MiriLRSSlitless`
- `MiriWFSS`

Right now, the only difference is the list of apertures that are plotted on the final scene, but this structure allows for future expansion while still sharing common tools.

In [ ]:
from whippot.modes import miri_lrs_tools

### Set up the initialization dictionary

We will demonstrate looking at the same field with all three modes. We will create a template for the initialization dictionary, and adjust entries as appropriate for each observing mode. The initialization dictionary is the same as for all WHIPPOT `ComputePositions` instances.

First, let's format the source positions to make it easier to copy and paste them into the initialization dictionary or GUI (this can also be done programatically):

In [ ]:
for s, c in sources.items():
    print(f"{s}: ({c.ra.deg:0.7f}, {c.dec.deg:0.7f})")

Fill out the standard initialization dictionary, copy-pasting the field into the `other_stars` key. Let's leave the `sci_aper` field blank for now. We'll set it separately for each LRS mode.

In [ ]:
initial_values={
    'instr': 'miri',
    'pa': 290.,
    'sci_ra': sources['SCI'].ra.deg, 'sci_dec': sources['SCI'].dec.deg,
}
# add a multi-line string of the other stars, copied from the cell above
initial_values['other_stars'] = """
a: (81.1097487, -70.0895429)
b: (81.1194942, -70.0901246)
c: (81.1509358, -70.0911632)
d: (81.1067000, -70.0837944)
"""

### LRS Slit

What about the LRS Slit? Let's put our target star `SCI` into the slit.

In [ ]:
# Use the MIRIM_SLIT aperture
initial_values.update({"sci_aper": "MIRIM_SLIT"})

cp1 = miri_lrs_tools.MiriLRSSlit(initial_values=initial_values)
cp1.ui

Well, one spectrum falls in the slit, but the others don't. This sounds like a case for Slitless spectroscopy.

### LRS Slitless

The LRS Slitless mode doesn't have any obscurations in the field of view. Maybe this will work?

In [ ]:
# Use the MIRIM_SLIT aperture
initial_values.update({"sci_aper": "MIRIM_SLITLESSPRISM"})

cp2 = miri_lrs_tools.MiriLRSSlitless(initial_values=initial_values)
cp2.ui

No, we still can only fit one of the spectra! LRS Slitless mode is designed for fast readout of a single bright target. We need something with a wider field of fiew. Maybe Wide-field Slitless Spectroscopy?

### Wide-field Slitless Spectroscopy

WFSS is implemented in the `MiriWFSS` class in `miri_lrs_tools.py`. This uses the ILLUM aperture, which is the illuminated part of the full detector array (can also use FULL, which shares a reference position).

In [ ]:
# Use the MIRIM_SLIT aperture
initial_values.update({"sci_aper": "MIRIM_ILLUM"})

cp3 = miri_lrs_tools.MiriWFSS(initial_values=initial_values)
cp3.ui

Heh, that looks more promising! Still, though, some of the sources are off the detector. Can we fix that by adjusting the PA and X/Y offsets? 

Note: I found the values below by using the GUI to explore the parameter space, but they are entered here using the initialization dictionary for demonstration purposes.

Looking at the detector scene suggests starting by moving the target star to the top of the detector by applying a y-offset of ~40 arcsec.

In [ ]:
initial_values.update({"final_idl_y": 40})
cp4 = miri_lrs_tools.MiriWFSS(initial_values=initial_values)
cp4.ui

OK, that's better, but we aren't taking advantage of the width of the detector. Let's move the SCI target to the top right corner with an additional X offset of 25 arcsec, and apply a rotation so that source c moves toward the bottom left corner. Maybe 45 degrees clockwise, so PA = 335?

In [ ]:
initial_values.update({"final_idl_x": 25, 'pa': 335})
cp5 = miri_lrs_tools.MiriWFSS(initial_values=initial_values)
cp5.ui

Oops, too much! Source c has rotated off of the illuminated region. Let's back off 5 degrees and shift a little further to the right:

In [ ]:
initial_values.update({'final_idl_x': 30, 'pa': 330})
cp6 = miri_lrs_tools.MiriWFSS(initial_values=initial_values)
cp6.ui

That looks great! Now when I define my program in APT, I will add a special requirement offset of (30, 40) arcsec, and an allowed V3PA range of 327-333 degrees.